In [1]:
import numpy as np
import pandas as pd
import math
from sympy import primerange
from scipy.stats import entropy

# --- Constants and configurations ---
alpha = 1 / 137.035999              # Fine structure constant (a)
alpha_inv = 137.035999             # Inverse of fine structure constant
phi = (1 + math.sqrt(5)) / 2
pi = math.pi
sqrt2 = math.sqrt(2)
e = math.e

# Digit windows for alpha variants
alpha_digit_signatures = [
    "7297", "297", "36034", "0036", "137035", "000137", "29735", "3603", "035"
]

# Scaling factors for nested testing
scaling_factors = [1, 3, 9, 27, 81, 137, 729, 1e-1, 1e-3, 1e-6]

# Fibonacci word (symbolic sequence)
def fibonacci_word(n):
    a, b = "0", "01"
    while len(b) < n:
        a, b = b, b + a
    return b[:n]

# Convert to decimal windows
def binary_to_decimal_windowed(bitstring, precision=12):
    values = []
    for i in range(len(bitstring) - precision):
        window = bitstring[i:i+precision]
        val = sum(int(b) / (2**(j+1)) for j, b in enumerate(window))
        values.append(val)
    return values

# Shannon entropy of decimal windows
def entropy_over_windows(values, window_size=8):
    entropy_vals = []
    for i in range(len(values) - window_size):
        window = values[i:i+window_size]
        freqs = np.histogram(window, bins=10, range=(0, 1), density=True)[0]
        if np.any(freqs):
            entropy_vals.append(entropy(freqs, base=2))
        else:
            entropy_vals.append(0)
    return entropy_vals

# Match digit substrings to alpha signature variants
def alpha_substring_match(val, precision=12):
    val_str = f"{val:.{precision}f}".replace(".", "")
    match_score = sum(1 for sig in alpha_digit_signatures if sig in val_str)
    return match_score

# Build result table
fib_str = fibonacci_word(150)
decimal_vals = binary_to_decimal_windowed(fib_str, precision=12)
entropy_vals = entropy_over_windows(decimal_vals, window_size=8)

# Build row-by-row
records = []
for i, base_val in enumerate(decimal_vals):
    row = {
        "original_val": base_val,
        "entropy": entropy_vals[i] if i < len(entropy_vals) else None
    }
    for scale in scaling_factors:
        scaled = base_val * scale
        inverse = 1 / scaled if scaled != 0 else 0

        row[f"scaled_{scale:.0e}"] = scaled
        row[f"match_scaled_{scale:.0e}"] = alpha_substring_match(scaled)

        row[f"inverse_{scale:.0e}"] = inverse
        row[f"match_inverse_{scale:.0e}"] = alpha_substring_match(inverse)

    records.append(row)

df_nested_alpha = pd.DataFrame(records)

# Display in Colab
from IPython.display import display
display(df_nested_alpha)


,original_val,entropy,scaled_1e+00,match_scaled_1e+00,inverse_1e+00,match_inverse_1e+00,scaled_3e+00,match_scaled_3e+00,inverse_3e+00,match_inverse_3e+00,...,inverse_1e-01,match_inverse_1e-01,scaled_1e-03,match_scaled_1e-03,inverse_1e-03,match_inverse_1e-03,scaled_1e-06,match_scaled_1e-06,inverse_1e-06,match_inverse_1e-06
0,0.290039,2.25,0.290039,0,3.447811,0,0.870117,0,1.149270,0,...,34.478114,0,0.000290,0,3447.811448,0,2.900391e-07,0,3.447811e+06,0
1,0.580322,2.25,0.580322,0,1.723180,0,1.740967,0,0.574393,0,...,17.231805,0,0.000580,0,1723.180480,0,5.803223e-07,0,1.723180e+06,0
2,0.160645,2.25,0.160645,0,6.224924,0,0.481934,0,2.074975,0,...,62.249240,0,0.000161,0,6224.924012,0,1.606445e-07,0,6.224924e+06,0
3,0.321533,2.25,0.321533,0,3.110099,0,0.964600,0,1.036700,0,...,31.100987,0,0.000322,0,3110.098709,0,3.215332e-07,0,3.110099e+06,0
4,0.643066,2.25,0.643066,0,1.555049,0,1.929199,0,0.518350,0,...,15.550494,0,0.000643,0,1555.049355,0,6.430664e-07,0,1.555049e+06,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133,0.161133,NaN,0.161133,0,6.206061,0,0.483398,0,2.068687,0,...,62.060606,0,0.000161,0,6206.060606,0,1.611328e-07,0,6.206061e+06,0
134,0.322510,NaN,0.322510,0,3.100681,0,0.967529,0,1.033560,0,...,31.006813,0,0.000323,0,3100.681302,0,3.225098e-07,0,3.100681e+06,0
135,0.645020,NaN,0.645020,0,1.550341,0,1.935059,0,0.516780,0,...,15.503407,0,0.000645,0,1550.340651,0,6.450195e-07,0,1.550341e+06,0
136,0.290039,NaN,0.290039,0,3.447811,0,0.870117,0,1.149270,0,...,34.478114,0,0.000290,0,3447.811448,0,2.900391e-07,0,3.447811e+06,0
